# Selection of HST images that have enough pixels
v >=26dec25

In [ ]:
import regex as re
import glob
import json
import yaml
import os
from types import SimpleNamespace
import galsim

In [ ]:
file_pattern = re.compile(r".*?(\d+).*?")


def get_order(file):
    match = file_pattern.match(os.path.basename(file))
    if not match:
        return math.inf
    return int(match.groups()[-1])


def get_list(loc, pattern):
    # dir = pathlib.Path(dir)
    a = list(glob.glob(loc + "/" + pattern))
    return sorted(a, key=get_order)


In [ ]:
#default config file
settings_dict = {
    "input_root_dir":"/lustre/fsn1/projects/rech/ixh/ufd72rp",  # root directory location of the dataset
    "input_dataset":"COSMOS_23.5_dataset"   # the dataset directory
}
args = SimpleNamespace(**settings_dict)

In [ ]:
args

In [ ]:
args.fov_pixels = 48  # minimal N pixels image

In [ ]:
args

In [ ]:
args.fn_out = "../list_gal_files_"+str(args.fov_pixels)+"pix.json"

In [ ]:
args

In [ ]:
# load origonal data
ds_path = args.input_root_dir + "/" + args.input_dataset + "/"
all_gal_o   = get_list(ds_path,"gal_*.fits")
all_psf_o   = get_list(ds_path,"psf_*.fits")
all_noise_o = get_list(ds_path,"noise_*.fits")
all_info_o  = get_list(ds_path,"info*.json")
n_total = len(all_gal_o)
assert n_total == len(all_psf_o), "pb n_total neq n_psf"
assert n_total == len(all_noise_o), "pb n_total neq n_noise"
assert n_total == len(all_info_o), "pb n_total neq n_info"

In [ ]:
n_total

In [ ]:
# filter original data to get at least fov_pixels x fov_pixels images
all_gal=[]
all_psf=[]
all_noise=[]
all_info=[]
for fgal,fpsf,fnoise,finfo in  zip(all_gal_o,all_psf_o,all_noise_o,all_info_o):
    gal_orig = galsim.fits.read(fgal).array
    if min(gal_orig.shape) >=args.fov_pixels:
        all_gal.append(fgal)
        all_psf.append(fpsf)
        all_noise.append(fnoise)
        all_info.append(finfo)
    

In [ ]:
len(all_gal)

In [ ]:
all_files = {"all_gal":all_gal,"all_psf":all_psf,"all_noise":all_noise,"all_info":all_info}

In [ ]:
try:
    with open(args.fn_out, "w") as file:
        json.dump(all_files, file, indent=4)
    print("List successfully saved to:",args.fn_out)
except IOError as e:
    print(f"Error writing to file: {e}")